In [1]:
import sys, time
import ray
from ray.tune import experiment
from ray.air import CheckpointConfig, RunConfig, session

from fran.managers.raytune_manager import *
from fran.utils.helpers import *

from skimage._shared.utils import check_nD
sys.path+= [".."]
import os; 
tr2 = ray.util.pdb.set_trace



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

https://app.neptune.ai/drusmanbashir/KiTS19/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api/project#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [2]:

# For myself. Delete this cell if you want
if is_notebook() == False:
    os.chdir("nbs")

In [3]:
common_paths_filename="config.yaml"
proj_defaults = load_proj_defaults(common_paths_filename,"kits21")
config_dict =  load_config_from_workbook(proj_defaults.configuration_filename, raytune=False)

# If you are new to ray-tune, please see their [Getting started](https://docs.ray.io/en/latest/tune/getting-started.html) page.
# Ray-tune options are set below. Alter them to your preference.

In [4]:
os.environ['TUNE_DISABLE_STRICT_METRIC_CHECKING']="1"
n_trials=5
raytune=True
profiler=False
n_gpu_per_trial=1
n_cpu_per_trial =32/n_gpu_per_trial    # alter this based on your number of cpu cores
    #Note: Make sure you have set the CUDA_VISIBLE_DEVICES env variable otherwise ray will not recognize your GPU.
neptune=True  # enables neptune logging.
max_epochs = 2
grace_period=1
if profiler==True:  
    os.environ["PERFTOOLS_PATH"]="/usr/lib/x86_64-linux-gnu/libprofiler.so"
    os.environ["PERFTOOLS_LOGFILE"]="/tmp/pprof.out"

config = load_config_from_workbook(proj_defaults.configuration_filename, raytune=raytune)
metadata =config['metadata']
dice_channel = 1 if out_channels_from_dict_or_cell(metadata['dest_labels'])==2 else 2
monitor = "loss_dice_region"+str(dice_channel)
# most_recent_experiment= metadata['most_recent_experiment'].item()

resume= False
most_recent_experiment="kits21_337"
if resume==True:
    experiment_name=most_recent_experiment
else:
    experiment_name=proj_defaults.project_title +"_"+ str(random.randint(1,1e+3)) 

debug_mode = False
if debug_mode ==True:
    ray.init(local_mode=True)
num_samples = 4 if debug_mode==True else n_trials
max_num_epochs = 3 if debug_mode==True else max_epochs
grace_period = max_num_epochs if debug_mode == True else grace_period
scheduler = ASHAScheduler(
    metric=monitor,
    mode="min",
    max_t=max_num_epochs,
    grace_period=grace_period,
    reduction_factor=2)
    


loss_channels = out_channels_from_dict_or_cell(metadata['dest_labels'])
loss_cols = ["loss_dice_label"+str(indx) for indx in range(1,loss_channels)]
# loss_cols = ["loss_dice_label1"]
loss_cols = ["training_iteration", "loss_ce_focal"] + loss_cols
reporter = CLIReporter(
    metric_columns=loss_cols ,
    parameter_columns = {'dataset_params/stride_random': 'random stride'},
)

In [5]:

train_single_gpu= partial(train_with_tune,False,neptune,max_num_epochs,proj_defaults)
train_multi_gpu= partial(train_with_tune,True, neptune,raytune,max_num_epochs,proj_defaults)
train = train_multi_gpu if n_gpu_per_trial>1 else train_single_gpu

# Setting up Ray Tuners: One for new trials. Another to resume previous trial (name stored manually as most_recent_experiment)

In [6]:


trial_path = proj_defaults.checkpoints_parent_folder/experiment_name
trainable_with_resources = tune.with_resources(train, {"cpu": 24, "gpu":1})
tuner_new = tune.Tuner(
    trainable_with_resources,
    tune_config=tune.TuneConfig(
        num_samples = num_samples,
        scheduler = scheduler,
        trial_name_creator=lambda trial: trial_str_creator(experiment_name), # this ignore trial name!
        trial_dirname_creator = trial_dirname_creator,
    ),
    param_space = config,
    run_config = RunConfig(
        name=experiment_name,

        local_dir = proj_defaults.checkpoints_parent_folder,
        checkpoint_config= CheckpointConfig(

        num_to_keep=5,
        checkpoint_at_end=False,
        ),
        progress_reporter=reporter
        
    ),

            

)

tuner_resume = tune.Tuner.restore(
    path=str(trial_path)

)

# Running the Ray Tune experiment. Note down the experiment name for reference. It is also stored under the corresponding online Neptune run as metadata/run_name

In [7]:


if resume==True:
    print("Resuming existing experiment, name: {}".format(experiment_name))
    results = tuner_resume.fit()
else:
    results = tuner_new.fit()

# 

2023-01-17 18:22:47,790	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(pid=563285) 
(pid=563285) 
(pid=563285) Please cite the following paper when using nnUNet:
(pid=563285) 
(pid=563285) Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z
(pid=563285) 
(pid=563285) 
(pid=563285) If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet
(pid=563285) 
== Status ==
Current time: 2023-01-17 18:22:49 (running for 00:00:00.17)
Memory usage on this node: 13.9/62.6 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 2.000: None | Iter 1.000: None
Resources requested: 24.0/48 CPUs, 1.0/2 GPUs, 0.0/31.51 GiB heap, 0.0/15.76 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /s/checkpoints/kits21/kits21_162
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+----------------+----------+---------------------+-----------------+
| Trial name     | status   | loc           

(func pid=563285) /home/ub/Dropbox/code/fran/fran/utils/callbacks.py:145: NeptuneDeprecationWarning: Parameter `run` is deprecated, use `with_id` instead. We'll end support of it in `neptune-client==1.0.0`.
(func pid=563285)   nep_run = neptune.init_run(run=None,mode="async",**self.neptune_run_settings)


(pid=563461) https://app.neptune.ai/drusmanbashir/KiTS19/
(func pid=563461) Loading dataset from BBox file /home/ub/datasets/preprocessed/kits21/stage2_patches/64_160_160/bboxes_info
(func pid=563461) Loading dataset from BBox file /home/ub/datasets/preprocessed/kits21/stage2_patches/64_160_160/bboxes_info
(func pid=563461) https://app.neptune.ai/drusmanbashir/KiTS19/
(func pid=563285) https://app.neptune.ai/drusmanbashir/KiTS19/e/KITS-2387


(func pid=563285) /home/ub/programs/miniconda3/envs/ai/lib/python3.10/site-packages/neptune/new/attributes/attribute.py:64: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` instead.
(func pid=563285)   return self.assign(value, wait)


...

(func pid=563285) /home/ub/programs/miniconda3/envs/ai/lib/python3.10/site-packages/neptune/new/attributes/atoms/string.py:54: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `String(str(object))` instead.
(func pid=563285)   value = StringVal(value)


Epoch 1/2 : |███████████████████████████████████-----| 88.89% [48/54 00:29<00:03 0.1511]
Epoch 1/2 : |████████████████████████████████████----| 90.74% [49/54 00:29<00:03 0.1491]
Epoch 1/2 : |█████████████████████████████████████---| 92.59% [50/54 00:30<00:02 0.1457]
Epoch 2/2 : |----------------------------------------| 0.00% [0/54 00:00<?]
Epoch 1/2 : |█████████████████████████████████████---| 94.44% [51/54 00:31<00:01 0.1435]
Epoch 1/2 : |██████████████████████████████████████--| 96.30% [52/54 00:31<00:01 0.1395]
Epoch 2/2 : |----------------------------------------| 1.85% [1/54 00:01<00:53]
Epoch 1/2 : |███████████████████████████████████████-| 98.15% [53/54 00:31<00:00 0.1359]
Epoch 1/2 : |----------------------------------------| 0.00% [0/14 00:00<?]              
Epoch 1/2 : |██--------------------------------------| 7.14% [1/14 00:00<00:01]
Epoch 1/2 : |█████-----------------------------------| 14.29% [2/14 00:00<00:01 0.1320]
Epoch 2/2 : |█--------------------------------------

(func pid=563461) /home/ub/programs/miniconda3/envs/ai/lib/python3.10/site-packages/neptune/new/attributes/atoms/string.py:54: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `String(str(object))` instead.
(func pid=563461)   value = StringVal(value)


Epoch 2/2 : |█████-----------------------------------| 12.96% [7/54 00:04<00:31 0.1376]
Epoch 2/2 : |█████-----------------------------------| 14.81% [8/54 00:05<00:32 0.1340]
Epoch 2/2 : |██████----------------------------------| 16.67% [9/54 00:06<00:30 0.1311]
Epoch 2/2 : |----------------------------------------| 0.00% [0/54 00:00<?]
Epoch 2/2 : |███████---------------------------------| 18.52% [10/54 00:06<00:28 0.1259]
Epoch 2/2 : |----------------------------------------| 1.85% [1/54 00:00<00:20]
Epoch 2/2 : |████████--------------------------------| 20.37% [11/54 00:07<00:28 0.1214]
Epoch 2/2 : |█---------------------------------------| 3.70% [2/54 00:01<00:34 0.1288]
Epoch 2/2 : |████████--------------------------------| 22.22% [12/54 00:07<00:26 0.1188]
Epoch 2/2 : |█████████-------------------------------| 24.07% [13/54 00:08<00:25 0.1144]
Epoch 2/2 : |██--------------------------------------| 5.56% [3/54 00:02<00:35 0.1316]
Epoch 2/2 : |██████████---------------------------

2023-01-17 18:26:51,445	INFO tune.py:762 -- Total run time: 242.13 seconds (241.91 seconds for the tuning loop).


# Examining the results: 
### Check out https://docs.ray.io/en/latest/tune/examples/tune_analyze_results.html for more info.

In [8]:
results

In [10]:
num_results = len(results)
print("Number of results:", num_results)

Number of results: 5


In [13]:
# Iterate over results
for i, result in enumerate(results):
    if result.error:
        print(f"Trial #{i} had an error:", result.error)
        continue

    print(
        f"Trial #{i} finished successfully with a mean accuracy metric of:",
        result.metrics["loss_dice_label2"]   # tumour dice
    )

Trial #0 finished successfully with a mean accuracy metric of: tensor(-0.2247)
Trial #1 finished successfully with a mean accuracy metric of: tensor(-0.3464)
Trial #2 finished successfully with a mean accuracy metric of: tensor(-0.2814)
Trial #3 finished successfully with a mean accuracy metric of: tensor(-0.3778)
Trial #4 finished successfully with a mean accuracy metric of: tensor(-0.3402)
